In [4]:
import pandas as pd
df_data = pd.read_excel(r'C:\Users\tomda\OneDrive\Desktop\Python Supper App\Python-ETL\Global_Superstore2.xlsx')
df_data.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,05-02-2013,07-02-2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,28-01-2013,30-01-2013,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,05-11-2013,06-11-2013,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [29]:

def transform_column_name(col):
    import re
    cleaned_col = re.sub(r'\s+', ' ',re.sub(r'[^a-zA-Z\s]', '', col)).strip().lower()
    cleaned_col = cleaned_col.replace(' ', '_')
    return cleaned_col
cols_sql = []
default_amount = ['amount', 'sales', 'quantity', 'discount', 'profit']
default_date = ['date']
j = 1
for i in df_data.columns:
    if transform_column_name(i) not in l:
        cols_sql.append(transform_column_name(i))
    else:
        cols_sql.append(transform_column_name(i) + '_' + str(j))
        j =+ 1
Amt_col = []
Date_col = []
for i in df_data.columns:
    for j in default_amount:
        if j in transform_column_name(i):
            Amt_col.append(i)
    for j in default_date:
        if j in transform_column_name(i):
            Date_col.append(i)
print(Amt_col)
print(Date_col)

['Sales', 'Quantity', 'Discount', 'Profit']
['Order Date', 'Ship Date']
